In [0]:
import requests
import os
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# Desabilita o aviso de segurança do SSL para evitar o erro 'CERTIFICATE_VERIFY_FAILED'
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# Define o caminho base do Volume
caminho_base_volume = "/Volumes/controlled_medication_data/bronze_layer/raw_data"

# Define o intervalo de anos para o download
ano_inicial = 2014
ano_final = 2021

# Loop externo para iterar sobre os anos
for ano in range(ano_inicial, ano_final + 1):
    
    # Define o diretório de destino dinamicamente para cada ano
    diretorio_destino = os.path.join(caminho_base_volume, str(ano))

    # Cria o diretório do ano se ele não existir
    if not os.path.exists(diretorio_destino):
        os.makedirs(diretorio_destino, exist_ok=True)
        print(f"Diretório '{diretorio_destino}' criado com sucesso no Volume.")
    else:
        print(f"Diretório '{diretorio_destino}' já existe no Volume.")
    
    # Loop interno para iterar sobre os meses (1 a 12)
    for mes in range(1, 13):
        # Condição para parar em novembro no ano final (2021)
        if ano == ano_final and mes > 11:
            break
        
        # Formata o número do mês para ter dois dígitos (ex: 1 -> 01)
        mes_formatado = f"{mes:02d}"
        
        # Constrói o nome do arquivo e a URL com base no ano e mês
        ano_mes = f"{ano}{mes_formatado}"
        nome_arquivo = f"EDA_Industrializados_{ano_mes}.csv"
        url = f"https://dados.anvisa.gov.br/dados/SNGPC/Industrializados/{nome_arquivo}"
        caminho_salvar = os.path.join(diretorio_destino, nome_arquivo)

        # Adiciona a verificação se o arquivo já existe na pasta
        if os.path.exists(caminho_salvar):
            print(f"Arquivo {nome_arquivo} já existe em '{diretorio_destino}'. Pulando o download.")
            continue  # Pula para a próxima iteração do loop

        print(f"Baixando o arquivo: {nome_arquivo}...")
        
        try:
            # Faz a requisição HTTP, desabilitando a verificação SSL
            resposta = requests.get(url, verify=False)
            
            if resposta.status_code == 200:
                with open(caminho_salvar, 'wb') as f:
                    f.write(resposta.content)
                print(f"Arquivo {nome_arquivo} baixado com sucesso em '{caminho_salvar}'.")
            else:
                print(f"Erro ao baixar o arquivo {nome_arquivo}. Código de status: {resposta.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Ocorreu um erro na requisição para {nome_arquivo}: {e}")

print("\nProcesso de download concluído.")